In [1]:
import shared.config as config
import shared.const as const
import pymongo
from pymongo import UpdateOne
import numpy as np
import pandas as pd
import json
import re
import ast

import sqlalchemy as db
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine, inspect
from sqlalchemy.engine import reflection

import logging
#create a logger
logger = logging.getLogger()

In [2]:
myclient = pymongo.MongoClient(str(config.MONGO_URI))
stagingDb = myclient[str(config.DB_STAGING)]
analyseDb = myclient[str(config.DB_ANALYSE)]
stagingCol = stagingDb[config.COLL_PLAATJES]
stagingOud = stagingDb[config.COLL_STAGING_OUD]
stagingNieuw = stagingDb[config.COLL_STAGING_NIEUW]
stagingMonster = stagingDb[config.COLL_STAGING_MONSTER]

analyseCol = analyseDb[config.COLL_ANALYSE]
analyseColClean = analyseDb[config.COLL_ANALYSE_CLEAN]

metaCollection = stagingDb['Kolominformatie']
AIRFLOW_WASSTRAAT_CONFIG = "./wasstraat_config/Wasstraat_Config_Harmonize.xlsx"




In [3]:
df_monsters = pd.DataFrame(list(analyseCol.find({'soort': 'Monster', 'brondata.PROJECT': {'$exists': True}}, {'projectcd':0})))
df_monsters.head(5)

set(list(df_monsters['project']))

{'ALT',
 'BAG114',
 'BAR',
 'BM',
 'BR96',
 'BTP',
 'BW84',
 'CPZ',
 'DB001',
 'DB197',
 'DC 16',
 'DC149',
 'DC150',
 'DC152',
 'DC154',
 'DE',
 'DL88',
 'DLKL05',
 'DLLH1',
 'DLLH2',
 'DLUT',
 'DT94',
 'EL91',
 'EZ05',
 'GROZO',
 'HGK',
 'HUY95',
 'IHE',
 'IPSE',
 'KEEN-III',
 'KV87',
 'KVII',
 'KVIII',
 'KVIV',
 'MAR6',
 'MASE',
 'MB2000',
 'MD005',
 'MD052',
 'MD056',
 'MD22-16',
 'MDHP03',
 'MDHP06',
 'MDHP07',
 'MDHP08',
 'MDHP12',
 'MNK04',
 'NAAK04',
 'NLD22-28',
 'OD116',
 'OD8',
 'OE171',
 'OLD1',
 'ONG',
 'PB',
 'PH24',
 'PNHK',
 'PNO3',
 'PO',
 'POL',
 'PPG',
 'PZPD2',
 'PZPD3',
 'PZPD4',
 'RV54',
 'SCHE',
 'SK85',
 'SOLL',
 'TP',
 'TUB',
 'UKL3',
 'VDA',
 'VOL21',
 'VOPO05',
 'VOPO06',
 'VOPO07',
 'VOPO09',
 'VRS',
 'VRS22',
 'WATLAB',
 'WH4',
 'WH7',
 'WL085',
 'WL163',
 'WLO17',
 'WP37',
 'ZU',
 'ZUPO'}

In [4]:
df_project = pd.DataFrame(list(analyseCol.find({'soort': 'Project'}, {'projectcd':1, 'project': 1, '_id':0})))

#df2_project = pd.concat([df_project['projectcd'], df_project['projectcd']], axis=1, ignore_index=True)
#df2_project.columns = ['projectcd', 'project']

#df_project.dropna(subset='project', inplace=True)
#df3_project = pd.concat([df_project, df2_project], ignore_index=True)
#df_project = df3_project.drop_duplicates(subset='project')
#df_project

df_project[['project', 'projectcd']][df_project.project == 'IPSE']
set(list(df_project['project']))

{248894,
 250279,
 'ABT40',
 'ALT',
 'AS83',
 'AV93',
 'Abts01',
 'BAG',
 'BAG114',
 'BAR',
 'BB9',
 'BLOK',
 'BR96',
 'BT13',
 'BT35',
 'BTP',
 'BU',
 'BW',
 'BW21',
 'BW8/10',
 'BW84',
 'Bag42',
 'BrasBunk',
 'CEL',
 'CHO',
 'CIS 15991',
 'CPZ',
 'CR93',
 'ChR',
 'DB143',
 'DB146',
 'DB148',
 'DB149',
 'DB150',
 'DB151',
 'DB152',
 'DB153',
 'DB154',
 'DB155',
 'DB157',
 'DB158',
 'DB159',
 'DB160',
 'DB161',
 'DB163',
 'DB164',
 'DB166',
 'DB186',
 'DB187',
 'DB188',
 'DB190',
 'DB191',
 'DB192',
 'DB193',
 'DB194',
 'DB197',
 'DB199',
 'DB200',
 'DB201',
 'DB202',
 'DB203',
 'DB204',
 'DB205',
 'DB206',
 'DB207',
 'DB208',
 'DB209',
 'DB210',
 'DB211',
 'DB212',
 'DB213',
 'DB214',
 'DB215',
 'DC157',
 'DC158',
 'DC164',
 'DC167',
 'DC168',
 'DC169',
 'DC170',
 'DC171',
 'DC172',
 'DC179',
 'DC22',
 'DC223',
 'DC224',
 'DC225',
 'DC228',
 'DC229',
 'DC231',
 'DC232',
 'DC241',
 'DC242',
 'DC243',
 'DC244',
 'DC246',
 'DC247',
 'DC248',
 'DC249',
 'DC250',
 'DC251',
 'DC252',
 'DC25

In [5]:
df = df_monsters.merge(df_project, on=['project'], how='left')
set(list(df[df.projectcd.notnull()]['project']))

{'ALT',
 'BAG114',
 'BAR',
 'BR96',
 'BTP',
 'BW84',
 'CPZ',
 'DB197',
 'DE',
 'DL88',
 'DLKL05',
 'DT94',
 'EL91',
 'EZ05',
 'GROZO',
 'HGK',
 'HUY95',
 'IHE',
 'IPSE',
 'KV87',
 'KVII',
 'KVIII',
 'KVIV',
 'MAR6',
 'MASE',
 'MD005',
 'MD052',
 'MDHP03',
 'MDHP06',
 'MDHP07',
 'MDHP08',
 'MDHP12',
 'MNK04',
 'NAAK04',
 'NLD22-28',
 'OD116',
 'OD8',
 'OE171',
 'OLD1',
 'ONG',
 'PB',
 'PH24',
 'POL',
 'RV54',
 'SK85',
 'TP',
 'TUB',
 'UKL3',
 'VDA',
 'VOL21',
 'VOPO05',
 'VOPO06',
 'VOPO07',
 'VOPO09',
 'VRS',
 'VRS22',
 'WATLAB',
 'WH4',
 'WH7',
 'WL085',
 'WL163',
 'ZU',
 'ZUPO'}

In [6]:
def fixMonsterProjectcds():
    """
    Method to fix the old projectnames that are being used in the Monsterdatabase. Unknown projectcodes are set to "Unknowm"

    Old projectnames are matched with the data from DeltIT Opgravingen, where both old and new values are found.
    """
    
    try: 
        logger.info("Starting fix of old projectcodes of Monster Database...")
        myclient = pymongo.MongoClient(str(config.MONGO_URI))
        analyseDb = myclient[str(config.DB_ANALYSE)]
        analyseCol = analyseDb[config.COLL_ANALYSE]

        # Fisrt set all projeccd to Unknown
        analyseCol.update_many({'soort': 'Monster'}, { "$set": { "projectcd": "Unknown" } })

        #Then get all Monsters with an old code
        df_monsters = pd.DataFrame(list(analyseCol.find({'soort': 'Monster', 'brondata.PROJECT': {'$exists': True}}, {'projectcd':0})))        
        
        df_project = pd.DataFrame(list(analyseCol.find({'soort': 'Project'}, {'projectcd':1, 'project': 1, '_id':0})))
        df2_project = pd.concat([df_project['projectcd'], df_project['projectcd']], axis=1, ignore_index=True)
        df2_project.columns = ['projectcd', 'project']

        # Add projectcd and project as porject-field to make sure all occurences are matched
        df_project.dropna(subset=['project'], inplace=True)
        df_project = pd.concat([df_project, df2_project], ignore_index=True)
        df_project.drop_duplicates(subset=['project'], inplace=True)
        
        # Set alle projectcd of monster database
        df_monsters = df_monsters.merge(df_project, on=['project'], how='left')
        df_monsters['projectcd'] = df_monsters['projectcd'].fillna(value = const.ONBEKEND_PROJECT)
        
        #Some projects were not found: report these
        logger.warning(f"Not all old projectcodes of Monsterdatabase could be fixed. These could not be matched: {set(list(df_monsters[df_monsters.projectcd == const.ONBEKEND_PROJECT]['project']))}")
        
        if not df_monsters.empty:
            # Update soort documents 
            updates=[ pymongo.UpdateOne({'_id':row['_id']}, {'$set':{'projectcd': row['projectcd']}}) for index, row in df_monsters.iterrows()]  # 
            analyseCol.bulk_write(updates)
        else:
            logger.warning(f"trying to update empty dataframe of monsters to fix old projeccodes.")

    except Exception as err:
        msg = "Onbekende fout bij het old projectcodes of Monster Database met melding: " + str(err)
        logger.error(msg)    
        raise Exception(msg) from err
    finally:
        myclient.close()


In [7]:
fixMonsterProjectcds()

Not all old projectcodes of Monsterdatabase could be fixed. These could not be matched: {'DLUT', 'KEEN-III', 'MB2000', 'PZPD2', 'DC 16', 'MD22-16', 'WP37', 'SCHE', 'PNO3', 'BM', 'PZPD4', 'PZPD3', 'PNHK', 'PPG', 'SOLL', 'WLO17', 'DLLH2', 'DLLH1', 'PO'}


In [8]:
aggr = [{'$match': {'soort': 'Monster', 'projectcd': {'$exists': True}}}, 
        {'$addFields': {'key_doos': {'$concat': ['P', {'$toString': '$projectcd'}, 'D', {'$toString': '$doosnr'}]}}}, 
        {'$addFields': {'key_project': {'$concat': ['P', {'$toString': '$projectcd'}]}}}, 
        {'$addFields': {'key_put': {'$concat': ['P', {'$toString': '$projectcd'}, {'$concat': ['P', {'$toString': '$putnr'}]}]}}}, 
        {'$addFields': {'key': {'$concat': ['M', {'$toString': '$monstercd'}]}}}, 
        {'$addFields': {'key_spoor': {'$concat': ['P', {'$toString': '$projectcd'}, {'$ifNull': [{'$concat': ['P', {'$toString': '$putnr'}]}, '']}, {'$ifNull': [{'$concat': ['V', {'$toString': '$vlaknr'}]}, '']}, 'S', {'$toString': '$spoornr'}]}}}, 
        #{'$addFields': {'key_vondst': {'$concat': ['P', '$projectcd', {'$cond': ['$vondstkey_met_putnr', {'$concat': ['P', {'$toString': '$putnr'}]}, '']}, {'$concat': ['V', {'$toString': '$vondstnr'}]}]}}}
       ]


df = pd.DataFrame(list(analyseCol.aggregate(aggr)))
df.head(5)

,_id,brondata,projectcd,putnr,vondstnr,monstercd,determinatiedatum,omstandigheden,gezeefd_volume,zeefmaat,...,karakterisering,project,soort,key_doos,key_project,key_put,key,key_spoor,doosnr,spoornr
0,62fd0021f7d2becb9bc77130,"{'_id': 62fd0021f7d2becb9bc77130, 'Id': 1, 'PR...",DB004,62.0,26.0,ALT_p62_v26,11/15/06 00:00:00,monster uit greppel,0.4,0.25,...,"A/B : vochtig, onkruid",ALT,Monster,None,PDB004,PDB004P62,MALT_p62_v26,None,NaN,NaN
1,62fd0021f7d2becb9bc77131,"{'_id': 62fd0021f7d2becb9bc77131, 'Id': 2, 'PR...",DB004,62.0,30.0,ALT_p62_v30,08/16/04 00:00:00,monster uit sloot,0.5,0.50,...,B:,ALT,Monster,PDB004D22,PDB004,PDB004P62,MALT_p62_v30,None,22.0,NaN
2,62fd0021f7d2becb9bc77132,"{'_id': 62fd0021f7d2becb9bc77132, 'Id': 3, 'PR...",DB004,65.0,2.0,ALT_p65_v2,06/24/05 00:00:00,NaN,0.1,0.50,...,"Wv:onkruid, beetje cultuur (Cannabis, Reseda l...",ALT,Monster,PDB004D22,PDB004,PDB004P65,MALT_p65_v2,None,22.0,NaN
3,62fd0021f7d2becb9bc77133,"{'_id': 62fd0021f7d2becb9bc77133, 'Id': 6, 'PR...",DC042,1.0,6.0,BAG114_p1_v6,05/11/04 00:00:00,bp-4,1.0,0.50,...,A:,BAG114,Monster,PDC042D1,PDC042,PDC042P1,MBAG114_p1_v6,None,1.0,NaN
4,62fd0021f7d2becb9bc77134,"{'_id': 62fd0021f7d2becb9bc77134, 'Id': 7, 'PR...",DC042,1.0,8.0,BAG114_p1_v8,10/08/04 00:00:00,top laag beer,0.8,0.50,...,A:,BAG114,Monster,PDC042D1,PDC042,PDC042P1,MBAG114_p1_v8,None,1.0,NaN
